In [1]:
import logging

from time import time, sleep
from multiprocessing import Process, Manager
logging.basicConfig()

from kazoo.client import KazooClient

In [10]:
class Philosopher(Process):
    def __init__(self, root: str, id: int, fork_path: str, eat_seconds: int = 15, max_id: int = 5):
        super().__init__()
        self.url = f'{root}/{id}'
        self.root = root
        self.fork = fork_path
        self.id = id
        self.left_fork_id = id
        self.right_fork_id = id + 1 if id + 1 < max_id else 0
        self.eat_seconds = eat_seconds
        self.partner_id_left = id - 1 if id - 1 >= 0 else max_id-1
        self.partner_id_right = id + 1 if id + 1 < max_id else 0
        
    def run(self):
        zk = KazooClient()
        zk.start()
        # Для каждого философа определяем путь до стола и его вилок
        table_lock = zk.Lock(f'{self.root}/table', self.id)
        left_fork = zk.Lock(f'{self.root}/{self.fork}/{self.left_fork_id}', self.id)
        right_fork = zk.Lock(f'{self.root}/{self.fork}/{self.right_fork_id}', self.id)

        start = time()
        while time() - start < self.eat_seconds:
            
            # Запрещаем нескольким философам одновременно брать вилки, для этгого лочим стол
            with table_lock:
                # Если вилки свободны и соседи поели не меньше чем ты, то можешь взять вилки
                if len(left_fork.contenders()) == 0 and len(right_fork.contenders()) == 0 \
                        and counters[self.partner_id_right] >= counters[self.id] \
                        and counters[self.partner_id_left] >= counters[self.id]:
                    # Берём вилки
                    left_fork.acquire()
                    right_fork.acquire()
            # Если взял вилку, соизволь покушать
            if left_fork.is_acquired and right_fork.is_acquired:
                print(f'{self.id} философ: Я придумал как взять вилки! Поем, пока не забыл...')
                counters[self.id] += 1
                # Пережёвывать еду очень важный и долгий процесс
                sleep(WAIT_EAT_MS)
                # Поел? Убери за собой (ну или хотя бы вилки положи)
                left_fork.release()
                right_fork.release() 
            else:
                print(f'{self.id} философ: Как бы взять вилки? Надо подумать...') # Нет вилок, так думай как их достать
            sleep(WAIT_AFTER_ALL_DONE)
            
        print(f'\n{self.id} философ поел {counters[self.id]} раз')
        zk.stop()
        zk.close()

In [11]:
root = '/task1'
fork_path = 'forks'
seconds_eat = 20
WAIT_EAT_MS = 1.25
WAIT_AFTER_ALL_DONE = 0.4

master_zk = KazooClient()
master_zk.start()

if master_zk.exists(root):
    master_zk.delete(root, recursive=True)

master_zk.create(root)
master_zk.create('/task1/table')
master_zk.create('/task1/forks')
for i in range(0,5):
    master_zk.create(f'/task1/forks/{i}')

counters = Manager().list()
p_list = list()
for i in range(0, 5):
    p = Philosopher(root, i, fork_path, seconds_eat)
    counters.append(0)
    p_list.append(p)
    
for p in p_list: 
    p.start()

0 философ: Я придумал как взять вилки! Поем, пока не забыл...
1 философ: Как бы взять вилки? Надо подумать...
2 философ: Я придумал как взять вилки! Поем, пока не забыл...
3 философ: Как бы взять вилки? Надо подумать...
4 философ: Как бы взять вилки? Надо подумать...
1 философ: Как бы взять вилки? Надо подумать...
3 философ: Как бы взять вилки? Надо подумать...
4 философ: Как бы взять вилки? Надо подумать...
1 философ: Как бы взять вилки? Надо подумать...
3 философ: Как бы взять вилки? Надо подумать...
4 философ: Как бы взять вилки? Надо подумать...
1 философ: Как бы взять вилки? Надо подумать...
3 философ: Как бы взять вилки? Надо подумать...
4 философ: Я придумал как взять вилки! Поем, пока не забыл...
1 философ: Я придумал как взять вилки! Поем, пока не забыл...
0 философ: Как бы взять вилки? Надо подумать...
3 философ: Как бы взять вилки? Надо подумать...
2 философ: Как бы взять вилки? Надо подумать...
0 философ: Как бы взять вилки? Надо подумать...
3 философ: Как бы взять вилки? Н